In [21]:
import numpy as np
import pickle
from segment_anything.utils import transforms
device = "cuda"
import torch
from segment_anything.modeling import sam
from segment_anything import sam_model_registry

In [2]:
def transform_image(img):
    image = np.expand_dims(img, axis=2)
    image = np.repeat(255 - image, 3, axis=2)
    return image

In [4]:
with open("..\data\Kaggle\preprocessed\\amd_001.pkl", 'rb') as file:
    dict = pickle.load(file)

In [5]:
image = dict['images'][30]['image']

In [6]:
transformed_image = transform_image(image)

In [7]:
transformed_image

array([[[ 67,  67,  67],
        [ 67,  67,  67],
        [ 69,  69,  69],
        ...,
        [ 31,  31,  31],
        [ 34,  34,  34],
        [ 32,  32,  32]],

       [[ 56,  56,  56],
        [ 57,  57,  57],
        [ 57,  57,  57],
        ...,
        [ 21,  21,  21],
        [ 24,  24,  24],
        [ 24,  24,  24]],

       [[ 56,  56,  56],
        [ 55,  55,  55],
        [ 53,  53,  53],
        ...,
        [ 24,  24,  24],
        [ 25,  25,  25],
        [ 26,  26,  26]],

       ...,

       [[238, 238, 238],
        [178, 178, 178],
        [233, 233, 233],
        ...,
        [223, 223, 223],
        [255, 255, 255],
        [184, 184, 184]],

       [[187, 187, 187],
        [201, 201, 201],
        [202, 202, 202],
        ...,
        [194, 194, 194],
        [255, 255, 255],
        [168, 168, 168]],

       [[237, 237, 237],
        [203, 203, 203],
        [255, 255, 255],
        ...,
        [207, 207, 207],
        [196, 196, 196],
        [171, 171, 171]]

In [10]:
transform = transforms.ResizeLongestSide(1000)

In [12]:
image_transformed = transform.apply_image(transformed_image)

In [15]:
image_torched = torch.as_tensor(image_transformed, device=device)

In [22]:
sam_checkpoint = "../data/SAM_checkpoint/sam_vit_b_01ec64.pth"
model_type = "vit_b"
device = "cuda"
sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_model.to(device=device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


In [23]:
sam_object = sam.Sam(sam_model.image_encoder,sam_model.prompt_encoder, sam_model.mask_decoder)

In [24]:
image_preprocessed = sam_object.preprocess(image_torched)

RuntimeError: The size of tensor a (512) must match the size of tensor b (3) at non-singleton dimension 0

In [26]:
image_torched.size()

torch.Size([512, 1000, 3])

In [27]:
input_image_torch = image_torched.permute(2, 0, 1).contiguous()[None, :, :, :]

In [29]:
input_image_torch.size()

torch.Size([1, 3, 512, 1000])

In [31]:
sam_object.to(device=device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


In [32]:
image_preprocessed = sam_object.preprocess(input_image_torch)

In [34]:
image_preprocessed

tensor([[[[-0.9705, -0.9705, -0.9363,  ...,  0.0000,  0.0000,  0.0000],
          [-1.1589, -1.1418, -1.1418,  ...,  0.0000,  0.0000,  0.0000],
          [-1.1589, -1.1760, -1.2103,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-0.8627, -0.8627, -0.8277,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0553, -1.0378, -1.0378,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0553, -1.0728, -1.1078,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-0.6367, -0.6367, -0.6018,  ...,  0.0000,  0.0000,  0.0000],
          [-0.8284, -0.8110, -

In [35]:
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters()) 

In [36]:
loss_fn = torch.nn.MSELoss()

In [38]:
with torch.no_grad():
    image_embedding = sam_model.image_encoder(image_preprocessed)

In [45]:
box = dict['images'][30]['bbox'][0]

In [74]:
torched_box = torch.as_tensor(box, device=device)

In [75]:
torched_box

tensor([  0., 259.,  66., 859.], device='cuda:0', dtype=torch.float64)

In [79]:
torched_box[None,:].float()

tensor([[  0., 259.,  66., 859.]], device='cuda:0')

In [73]:
transform.apply_boxes_torch(torched_box, (512,1000))

tensor([[  0., 259.,  66., 859.]], device='cuda:0')

In [77]:
with torch.no_grad():
    sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
        points=None,
        boxes=transform.apply_boxes_torch(torched_box, (512,1000)),
        masks=None,
    )

In [46]:
box

array([  0., 259.,  66., 859.])

In [50]:
low_res_masks, iou_predictions = sam_model.mask_decoder(
    image_embeddings=image_embedding,
    image_pe=sam_model.prompt_encoder.get_dense_pe(),
    sparse_prompt_embeddings=sparse_embeddings,
    dense_prompt_embeddings=dense_embeddings,
    multimask_output=False,
)

In [52]:
low_res_masks.size()

torch.Size([1, 1, 256, 256])

In [53]:
upscaled_masks = sam_model.postprocess_masks(low_res_masks, (256,256), (512,1000)).to(device)

from torch.nn.functional import threshold, normalize

binary_mask = normalize(threshold(upscaled_masks, 0.0, 0)).to(device)

In [55]:
binary_mask.size()

torch.Size([1, 1, 512, 1000])

In [56]:
gt_binary_mask = dict['images'][30]['segmentations'][0]

In [58]:
gt_binary_mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
torched_gt = torch.as_tensor(gt_binary_mask, device=device)

In [62]:
torched_gt[None,None,:,:].size()

torch.Size([1, 1, 512, 1000])

In [64]:
gt = torched_gt[None,None,:,:]

In [71]:
loss = loss_fn(binary_mask, gt.float())

In [72]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [70]:
gt.float()

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')